In [ ]:
pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

# 1. BitsAndBytes 4bit config 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",               # nf4: 높은 정밀도
    bnb_4bit_use_double_quant=True,          # 더 나은 표현력
    bnb_4bit_compute_dtype=torch.float16     # 연산은 FP16
)

# 2. 모델 로드
model_name = "ChanceFocus/finma-7b-full"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. 저장할 디렉토리 지정
save_dir = "./finma-7b-4bit"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# 4. 저장된 모델 디렉토리 크기 확인 함수
def get_dir_size_mb(path):
    total = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            if os.path.isfile(fp):
                total += os.path.getsize(fp)
    return total / 1024**2  # MB 단위

print(f"Model saved to: {save_dir}")
print(f"Model size: {get_dir_size_mb(save_dir):.2f} MB")

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:609: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly as `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation
  warnings.warn(


Model saved to: ./finma-7b-4bit
Model size: 3690.88 MB


In [ ]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import login, create_repo, upload_folder

# 2. 업로드할 경로 설정
local_model_dir = "./finma-7b-4bit"
repo_id = "capston-team-5/finma-7b-4bit-quantized"  # 원하는 모델 이름으로 변경

# 3. Hugging Face에 저장소 생성 (이미 있으면 skip=True로)
create_repo(repo_id, private=False, exist_ok=True)

# 4. 모델 폴더 전체 업로드
upload_folder(
    repo_id=repo_id,
    folder_path=local_model_dir,
    path_in_repo="",     # 루트에 업로드
    commit_message="Upload 4bit quantized finma-7b model"
)

print(f"✅ Model uploaded to: https://huggingface.co/{repo_id}")


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

✅ Model uploaded to: https://huggingface.co/capston-team-5/finma-7b-4bit-quantized
